In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib nbagg
import librosa
import jams
import librosa.display
import jams.display
import scipy
import resampy
from IPython.display import Audio

import mir_eval

import pandas as pd

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
def resample_features(X, framerate=0.25, hop_length=512, sr=22050, **kwargs):
    
    return resampy.resample(X, sr/float(hop_length), 1./framerate, **kwargs)

In [4]:
def selfsim(x, metric='sqeuclidean'):
    
    D = scipy.spatial.distance.cdist(x.T, x.T, metric=metric)
    bw = np.mean(np.median(D, axis=1))
    
    return np.exp(-D / bw)

def simsim(A, B):
    '''Calculates the cross-correlation between two square matrices A and B'''
    n = min(len(A), len(B))
    
    A = A[:n, :n] - np.mean(A[:n, :n])
    B = B[:n, :n] - np.mean(B[:n, :n])
    
    anorm = np.sqrt(np.sum(A[:n,:n]**2))
    bnorm = np.sqrt(np.sum(B[:n,:n]**2))
    
    return np.sum(A[:n, :n] * B[:n, :n]) / (anorm * bnorm)

In [3]:
from joblib import Parallel, delayed

In [4]:
from tqdm import tqdm

In [5]:
import os

In [25]:
def compute_features(audio_file, outdir, framerate=0.25, salami=True):
    
    y, sr = librosa.load(audio_file)
    
    
    chroma = librosa.feature.stack_memory(resample_features(librosa.feature.chroma_cqt(y=librosa.effects.harmonic(y, margin=8.0),
                                                                                   sr=sr,
                                                                                   tuning=0,
                                                                                   bins_per_octave=36),
                                                           framerate=framerate),
                                      n_steps=3)
    
    mfcc = librosa.feature.stack_memory(resample_features(librosa.feature.mfcc(y=y, sr=sr),
                                                          framerate=framerate),
                                        n_steps=3)
    
    tgram = librosa.feature.tempogram(y=y, sr=sr)
    rgram = np.abs(librosa.fmt(tgram, axis=0, n_fmt=128, t_min=0.25))

    tgram = resample_features(tgram, framerate=framerate)
    rgram = resample_features(rgram, framerate=framerate)
    
    if salami:
        outfile = os.path.join(outdir, os.extsep.join([os.path.basename(os.path.dirname(audio_file)), 'npz']))
    else:
        outfile = os.path.join(outdir, os.extsep.join([os.path.basename(audio_file), 'npz']))
        
    np.savez(outfile, chroma=chroma, mfcc=mfcc, tempo=tgram, rhythm=rgram)

In [7]:
files = librosa.util.find_files('/home/bmcfee/data/salami-data-public/audio/')

In [13]:
!mkdir -p /home/bmcfee/working/segment_labels_features/{SALAMI,SPAM}

In [24]:
Parallel(n_jobs=20, verbose=1)(delayed(compute_features)(af, '/home/bmcfee/working/segment_labels_features/SALAMI/')
                               for af in files)

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   56.7s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed: 10.9min
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed: 27.3min
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed: 45.8min
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed: 70.5min
[Parallel(n_jobs=20)]: Done 1446 out of 1446 | elapsed: 84.7min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [30]:
files = librosa.util.find_files('/home/bmcfee/data/SPAM/')

In [32]:
Parallel(n_jobs=20, verbose=1)(delayed(compute_features)(af, '/home/bmcfee/working/segment_labels_features/SPAM/', salami=False)
                               for af in files)

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:  3.7min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]